In [240]:
import ontospy
import pandas as pd
from ontodocs.viz.viz_html_single import *
import rdflib
import pronto
import inflection
import re
import numpy as np
from tqdm import *
import json
import os
import sys
import codecs
import collections
import nltk
from lxml import etree
from lxml.html.clean import Cleaner
from pymystem3 import Mystem
from nltk.corpus import stopwords
from keras_text.models import TokenModelFactory
from keras_text.models import YoonKimCNN, AttentionRNN, StackedRNN
from keras_text.processing import WordTokenizer
from keras_text.data import Dataset

ImportError: No module named 'keras_text.models'

# Анализ онтологии

Получаем элементы онтологии, которые являются терминальными и исходят из класса Wine

In [1]:
import rdflib
g = rdflib.Graph()
g.load('./wine.rdf')

Составляем фрагмент онтологии

In [61]:
classes = ["AlsatianWine", "AmericanWine", "Beaujolais", "Bordeaux", "Medoc", "Margaux", "Pauillac", "RedBordeaux", "Sauternes", "StEmilion", "WhiteBordeaux", "Burgundy", "RedBurgundy", "CotesDOr", "WhiteBurgundy", "Meursault", "CabernetFranc", "CabernetSauvignon", "CaliforniaWine", "Chardonnay", "CheninBlanc", "DessertWine", "IceWine", "SweetRiesling", "DryWine", "DryRedWine", "DryWhiteWine", "EarlyHarvest", "FrenchWine", "FullBodiedWine", "Gamay", "GermanWine", "ItalianWine", "Chianti", "LateHarvest", "IceWine", "Sauternes", "Loire", "Anjou", "Muscadet", "Sancerre", "Tours", "WhiteLoire", "Meritage", "Merlot", "PetiteSyrah", "PinotBlanc", "PinotNoir", "RedWine", "DryRedWine", "Port", "RedBordeaux", "RedBurgundy", "CotesDOr", "Riesling", "DryRiesling", "SweetRiesling", "RoseWine", "SemillonOrSauvignonBlanc", "SauvignonBlanc", "Semillon", "SweetWine", "TableWine", "RedTableWine", "WhiteTableWine", "TexasWine", "WhiteWine", "DryWhiteWine", "WhiteBordeaux", "WhiteBurgundy", "Meursault", "WhiteLoire", "WhiteNonSweetWine", "Zinfandel"]

In [62]:
whitelist_features = ["hasBody", "hasSugar", "hasFlavor", "hasColor"]
def find_features(graph, class_name):
    features = {}
    for s, p, o in graph:
        selected_class = str(s).split("#")[-1]
        selected_feature = str(p).split("#")[-1]
        if ((class_name == selected_class) and (selected_feature in whitelist_features)):
            features[selected_feature] = str(o).split("#")[-1]
    return features

def find_instances(graph, class_name):
    instances = []
    for s, p, o in graph:
        selected_class = str(o).split("#")[-1]
        selected_feature = str(p).split("#")[-1]
        if ((class_name == selected_class) and (selected_feature == "type")):
            instances.append(str(s).split("#")[-1])
    return instances

In [69]:
instances = [find_instances(g, c) for c in classes]
instances = [i for l in instances for i in l]

In [70]:
ontology = {i: find_features(g, i) for i in instances if find_features(g, i) != {}}

# Анализ отзывов

In [72]:
reviews = pd.read_csv("./wine-reviews/winemag-data-130k-v2.csv")

In [73]:
reviews

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
6,6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
7,7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
8,8,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,NaN,NaN,Anna Lee C. Iijima,NaN,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel
9,9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam


Какие сомелье участвовали

In [74]:
reviews.groupby("taster_name").describe()["points"][["count"]]

,count
taster_name,
Alexander Peartree,415.0
Anna Lee C. Iijima,4415.0
Anne Krebiehl MW,3685.0
Carrie Dykes,139.0
Christina Pickard,6.0
Fiona Adams,27.0
Jeff Jenssen,491.0
Jim Gordon,4177.0
Joe Czerwinski,5147.0


Какие вина описывались и как они соотносятся с онтологией

In [75]:
reviews.groupby("variety").describe()["points"][["count"]]

,count
variety,
Abouriou,3.0
Agiorgitiko,63.0
Aglianico,359.0
Aidani,1.0
Airen,3.0
Albana,23.0
Albanello,1.0
Albariño,477.0
Albarossa,2.0


In [76]:
types = [inflection.underscore(k).split("_") for k in ontology.keys()]

In [77]:
reviews.groupby("variety").describe()["points"][["count"]].sort_values(by=["count"])

,count
variety,
Žilavka,1.0
Orangetraube,1.0
Catalanesca,1.0
Ondenc,1.0
Ojaleshi,1.0
Centesimino,1.0
Cercial,1.0
Muscadel,1.0
Otskhanuri Sapere,1.0


Сопоставляем вина по названию

In [78]:
reviews["title_array"] = reviews["title"].apply(lambda x: re.sub("[\(\)\.-]", "", x)).str.lower().str.split()

In [79]:
def jaccard(x, y):
    x = list(set(x))
    y = list(set(y))
    intersection = list(set([a for a in x if a in y]))
    union = list(set(x + y))
    return len(intersection)/len(union)

In [124]:
reviews["max_jaccard"] = reviews["title_array"].apply(lambda x: np.max([jaccard(x, y) for y in types]))
reviews["closest_type"] = reviews["title_array"].apply(lambda x: types[np.argmax([jaccard(x, y) for y in types])])

In [82]:
reviews[reviews["max_jaccard"] > 0.4][["title_array", "closest_type", "max_jaccard"]].sort_values(by=["max_jaccard"], ascending=[0])

,title_array,closest_type,max_jaccard
120428,"[santa, cruz, mountain, vineyard, 2013, luches...","[santa, cruz, mountain, vineyard, cabernet, sa...",0.666667
36357,"[mount, eden, vineyards, 2006, wolff, vineyard...","[mount, eden, vineyard, edna, valley, chardonnay]",0.666667
117118,"[santa, cruz, mountain, vineyard, 2013, luches...","[santa, cruz, mountain, vineyard, cabernet, sa...",0.666667
22986,"[mount, eden, vineyards, 2010, wolff, vineyard...","[mount, eden, vineyard, edna, valley, chardonnay]",0.666667
124308,"[santa, cruz, mountain, vineyard, 2012, luches...","[santa, cruz, mountain, vineyard, cabernet, sa...",0.666667
106642,"[mount, eden, vineyards, 2010, wolff, vineyard...","[mount, eden, vineyard, edna, valley, chardonnay]",0.666667
74242,"[mount, eden, vineyards, 2012, wolff, vineyard...","[mount, eden, vineyard, edna, valley, chardonnay]",0.666667
108168,"[page, mill, winery, 2012, cabernet, sauvignon...","[page, mill, winery, cabernet, sauvignon]",0.625000
33767,"[mount, eden, vineyards, 2015, chardonnay, edn...","[mount, eden, vineyard, edna, valley, chardonnay]",0.625000
74753,"[mount, eden, vineyards, 2015, chardonnay, edn...","[mount, eden, vineyard, edna, valley, chardonnay]",0.625000


Составляем датасет

In [85]:
def get_features(x):
    features = []
    for k, v in x.items():
        features.append(k.replace("has", "") + "Is" + v)
    return features

In [103]:
features = list(set(f for k, v in ontology.items() for f in get_features(v)))

In [100]:
wines = list(ontology.keys())

In [104]:
features

['BodyIsMedium',
 'SugarIsOffDry',
 'BodyIsLight',
 'FlavorIsModerate',
 'BodyIsFull',
 'SugarIsSweet',
 'FlavorIsDelicate',
 'SugarIsDry',
 'ColorIsWhite',
 'FlavorIsStrong']

In [90]:
links = [(k, f) for k, v in ontology.items() for f in get_features(v)]

In [105]:
wine_nodes = [{"id": wine, "group": "wine"} for wine in wines]

In [106]:
feature_nodes = [{"id": feature, "group": "feature"} for feature in features]

In [107]:
nodes = wine_nodes + feature_nodes

In [110]:
graph_links = [{"source": wine, "target": feature} for wine, feature in links]

In [113]:
graph = {"nodes": nodes, "links": graph_links}

In [117]:
json.dumps(graph)

'{"nodes": [{"group": "wine", "id": "StonleighSauvignonBlanc"}, {"group": "wine", "id": "ChateauDYchemSauterne"}, {"group": "wine", "id": "ChiantiClassico"}, {"group": "wine", "id": "StGenevieveTexasWhite"}, {"group": "wine", "id": "MariettaZinfandel"}, {"group": "wine", "id": "KathrynKennedyLateral"}, {"group": "wine", "id": "CongressSpringsSemillon"}, {"group": "wine", "id": "FormanChardonnay"}, {"group": "wine", "id": "PulignyMontrachetWhiteBurgundy"}, {"group": "wine", "id": "LongridgeMerlot"}, {"group": "wine", "id": "SeanThackreySiriusPetiteSyrah"}, {"group": "wine", "id": "SelaksIceWine"}, {"group": "wine", "id": "MariettaCabernetSauvignon"}, {"group": "wine", "id": "MariettaOldVinesRed"}, {"group": "wine", "id": "CortonMontrachetWhiteBurgundy"}, {"group": "wine", "id": "WhitehallLanePrimavera"}, {"group": "wine", "id": "WhitehallLaneCabernetFranc"}, {"group": "wine", "id": "SaucelitoCanyonZinfandel1998"}, {"group": "wine", "id": "SantaCruzMountainVineyardCabernetSauvignon"}, {"

# Чистка отзывов

In [159]:
reviews["filtered_description"] = reviews["description"].str.lower().apply(lambda x: re.sub("[^\w\s]", "", x))

In [160]:
stopwords_list = set(stopwords.words('english'))

In [161]:
reviews["filtered_description"] = reviews["filtered_description"].str.lower().str.split().apply(lambda x: [w for w in x if w not in stopwords_list])

In [162]:
reviews["filtered_description"] = reviews["filtered_description"].apply(lambda x: [w for w in x if len(w) > 2])

In [190]:
mystem = Mystem()

In [191]:
def lemmatize_words(word_list):
    processed_word_list = []
    for word in word_list:
        word = mystem.lemmatize(word)[0]
        processed_word_list.append(word)
    return processed_word_list    

In [192]:
lemmas = []
for r in tqdm(reviews["filtered_description"]):
    lemmas.append(lemmatize_words(r))
reviews["filtered_description"] = lemmas

100%|██████████| 129971/129971 [11:17<00:00, 191.89it/s]


# Формирование выборки и классификация

In [361]:
from sklearn.model_selection import train_test_split

In [362]:
from keras.preprocessing import text
texts = selected_reviews["filtered_description"].apply(lambda x: " ".join(x)).tolist()

In [379]:
words = []
def get_word_index(w):
    if w not in words:
        words.append(w)
    return words.index(w)

In [380]:
from keras.preprocessing.text import Tokenizer
tk = Tokenizer()
tk.fit_on_texts(texts)
X = [[get_word_index(w) for w in text] for text in texts]

In [381]:
y = pd.get_dummies(selected_reviews, columns=["flavor"])[["flavor_Delicate", "flavor_Moderate", "flavor_Strong"]].as_matrix()

In [382]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

Преобразовать к bag of words каждое предложение

In [383]:
'''This scripts implements Kim's paper "Convolutional Neural Networks for Sentence Classification"
with a very small embedding size (20) than the commonly used values (100 - 300) as it gives better
result with much less parameters.

Run on GPU: THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python imdb_cnn.py

Get to 0.853 test accuracy after 5 epochs. 13s/epoch on Nvidia GTX980 GPU.
'''

from __future__ import print_function
import numpy as np
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Input, Add
from keras.layers.core import Dropout, Activation, Flatten, Dense
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D

# set parameters:
max_features = 5000  # vocabulary size
maxlen = 100  # maximum length of the review
batch_size = 32
embedding_dims = 20
ngram_filters = [3, 5, 7]
nb_filter = 1200  # number of filters for each ngram_filter
nb_epoch = 5

# prepare data
print('Loading data...')
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Loading data...
1898 train sequences
633 test sequences
Pad sequences (samples x time)
X_train shape: (1898, 100)
X_test shape: (633, 100)


In [387]:
input_layer =  Input(shape=(maxlen,))
embeddings = Embedding(max_features, embedding_dims, input_length=maxlen)(input_layer)
dropout = Dropout(0.)(embeddings)
outs = []
for n_gram in ngram_filters:
    convolution = Convolution1D(filters=nb_filter,
                                 kernel_size=n_gram,
                                 padding='valid',
                                 activation='relu',
                                 strides=1,
                                 input_shape=(embedding_dims, maxlen))(dropout)
    pooling = MaxPooling1D(pool_size=maxlen - n_gram + 1)(convolution)
    flatten = Flatten()(pooling)
    outs.append(flatten)
added = Add()(outs) 
dropout = Dropout(0.)(added)
dense = Dense(3, input_dim=nb_filter * len(ngram_filters))(dropout)
activation = Activation('sigmoid')(dense)

model = Model(inputs=input_layer, outputs=activation)

In [389]:
# train model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=nb_epoch,
          validation_data=(X_test, y_test),)
# acc = accuracy(y_test,
#                np.round(np.array(model.predict({'input': X_test},
#                                                batch_size=batch_size)['output'])))
# print('Test accuracy:', acc)

Train on 1898 samples, validate on 633 samples
Epoch 1/5
1898/1898 [==============================] - 81s 43ms/step - loss: 0.5186 - acc: 0.7443 - val_loss: 0.5064 - val_acc: 0.7130
Epoch 2/5
1898/1898 [==============================] - 92s 49ms/step - loss: 0.4433 - acc: 0.7764 - val_loss: 0.5477 - val_acc: 0.7541
Epoch 3/5
1898/1898 [==============================] - 118s 62ms/step - loss: 0.3973 - acc: 0.8112 - val_loss: 0.4647 - val_acc: 0.7878
Epoch 4/5
1898/1898 [==============================] - 105s 55ms/step - loss: 0.3622 - acc: 0.8267 - val_loss: 0.4354 - val_acc: 0.7878
Epoch 5/5
1898/1898 [==============================] - 78s 41ms/step - loss: 0.3333 - acc: 0.8456 - val_loss: 0.4044 - val_acc: 0.7936


In [196]:
reviews["closest_instance"] = reviews["title_array"].apply(lambda x: inflection.camelize("_".join(types[np.argmax([jaccard(x, y) for y in types])])))

In [227]:
selected_reviews = reviews[reviews["max_jaccard"] > 0.3]

Признаки, связанные с ароматом

In [229]:
selected_reviews["flavor"] = selected_reviews["closest_instance"].apply(lambda x: ontology[x]['hasFlavor'] if 'hasFlavor' in ontology[x].keys() else None)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [230]:
selected_reviews["flavor"].value_counts()

Moderate    1019
Strong       587
Delicate     336
Name: flavor, dtype: int64

# Создать онтологию на основе полученной модели